In [3]:
#import sys
#!{sys.executable} -m pip install --upgrade "scikit-learn>=1.6.1,<1.7" category_encoders shap lyft_fugue

In [2]:
#usual packages
import sklearn
import numpy as np
from IPython.display import IFrame
from sklearn.model_selection import GroupKFold
from datetime import date
import pandas as pd
from pandas import DataFrame
import subprocess
import sys

#lyft packages
import lyft_data_toolkit.db.write.df
from lyft_data_toolkit.db.read import Compute
from lyft_data_toolkit.db.read import Query
from lyft_data_toolkit.db.read.df import to_dataframe  
from lyft_data_toolkit.v1_to_v3 import hive
from lyft_data_toolkit.v1_to_v3 import presto
from lyft_data_toolkit.v1_to_v3 import spark
from lyft_data_toolkit.v1_to_v3 import trino

#other packages
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [4]:
#pull data (might be useless)
SQL ="""
--higherlimit
WITH pre_final_table AS (
SELECT DISTINCT 
       sub1.rider_lyft_id,
       sub1.business_program_user_id,
       sub1.business_program_type,
       sub1.business_program_type_cleaned,
       sub1.lyft_signed_up_at,
       sub1.bp_created_at,
       sub1.bp_activated_at,
       sub1.user_type,
       sub1.sub_user_type,
       sub2.first_ride_finished_at
  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1
  LEFT JOIN coco.dim_user sub2 
    ON sub1.rider_lyft_id = sub2.user_lyft_id
 WHERE sub1.is_business_ride = TRUE 
) 

,pre_final_table_v2 AS (
SELECT rider_lyft_id,
       business_program_user_id,
       business_program_type,
       business_program_type_cleaned,
       lyft_signed_up_at,
       bp_created_at,
       bp_activated_at,
       user_type,
       sub_user_type,
       first_ride_finished_at,
       ROW_NUMBER() OVER(PARTITION BY rider_lyft_id ORDER BY bp_activated_at DESC) rnk 
  FROM pre_final_table 
 WHERE TRUE
)

SELECT 
week(bp_activated_at) as week_num, -------new added
year(bp_activated_at) as year_num, -------new added
DATE_TRUNC('week',CAST(bp_activated_at AS DATE)) week,
       business_program_type_cleaned,
       CASE WHEN DATE_DIFF('day',first_ride_finished_at,bp_activated_at) > 0 THEN 'PP' ELSE 'NP' END AS activation_type, -- proxy for NP/PP
       coalesce(COUNT(DISTINCT rider_lyft_id),0) count_activations
  FROM pre_final_table_v2
 WHERE TRUE 
   AND rnk = 1
 GROUP BY 1,2,3,4,5
"""  
df = pd.DataFrame()
df = presto.query(SQL, scheduled=True)

{"ts": "2025-11-13T21:12:32.467Z", "uuid": "dccb1e4a-53e8-42e9-90ba-c86dff412dc2", "name": "lyft_data_toolkit.v1_to_v3.presto", "lvlname": "WARNING", "lvlno": 30, "pid": 568, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/presto.py:18", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: presto.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 567395937, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/567395937', 'query': "--higherlimit\nWITH pre_final_table AS (\nSELECT DISTINCT \n       sub1.rider_lyft_id,\n       sub1.business_program_user_id,\n       sub1.business_program_type,\n       sub1.business_program_type_cleaned,\n       sub1.lyft_signed_up_at,\n       sub1.bp_created_at,\n       sub1.bp_activated_at,\n       sub1.user_type,\n       sub1.sub_user_type,\n       sub2.first_ride_finished_at\n  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1\n  LEFT JOIN coco.dim_user sub2 \n    ON sub1.rider_lyft_id = sub2.user_lyft_id\n WHERE sub1.is_business_ride = TRUE \n) \n\n,pre_final_table_v2 AS (\nSELECT rider_lyft_id,\n       business_program_user_id,\n       business_program_type,\n       business_program_type_cleaned,\n       lyft_signed_up_at,\n       bp_created_at,\n       bp_activated_at,\n       user_type,\n       sub_us

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   week_num                       903 non-null    float64       
 1   year_num                       903 non-null    float64       
 2   week                           903 non-null    datetime64[ns]
 3   business_program_type_cleaned  903 non-null    object        
 4   activation_type                904 non-null    object        
 5   count_activations              904 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 42.5+ KB


In [6]:
df=df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 903 entries, 0 to 903
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   week_num                       903 non-null    float64       
 1   year_num                       903 non-null    float64       
 2   week                           903 non-null    datetime64[ns]
 3   business_program_type_cleaned  903 non-null    object        
 4   activation_type                903 non-null    object        
 5   count_activations              903 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 49.4+ KB


In [8]:
#pivot table to have columns for managed vs organic activations
df_pivot = df.pivot_table(
    index=["week_num","year_num","week", "activation_type"],
    columns="business_program_type_cleaned",
    values="count_activations",
    fill_value=0
).reset_index()
print(df_pivot['managed'].sum())
print(df_pivot['organic'].sum())
print(df_pivot)

136925
490061
business_program_type_cleaned  week_num  year_num       week activation_type  \
0                                   1.0    2022.0 2022-01-03              NP   
1                                   1.0    2022.0 2022-01-03              PP   
2                                   1.0    2023.0 2023-01-02              NP   
3                                   1.0    2023.0 2023-01-02              PP   
4                                   1.0    2024.0 2024-01-01              NP   
..                                  ...       ...        ...             ...   
447                                52.0    2023.0 2022-12-26              PP   
448                                52.0    2023.0 2023-12-25              NP   
449                                52.0    2023.0 2023-12-25              PP   
450                                52.0    2024.0 2024-12-23              NP   
451                                52.0    2024.0 2024-12-23              PP   

business_program_type_cle

In [9]:
#pivot table to have columns for managed vs organic activations
df_pivot = df.pivot_table(
    index=["week_num","year_num","week", "activation_type"],
    columns="business_program_type_cleaned",
    values="count_activations",
    fill_value=0
).reset_index()
print(df_pivot['managed'].sum())
print(df_pivot['organic'].sum())
print(df_pivot)

136925
490061
business_program_type_cleaned  week_num  year_num       week activation_type  \
0                                   1.0    2022.0 2022-01-03              NP   
1                                   1.0    2022.0 2022-01-03              PP   
2                                   1.0    2023.0 2023-01-02              NP   
3                                   1.0    2023.0 2023-01-02              PP   
4                                   1.0    2024.0 2024-01-01              NP   
..                                  ...       ...        ...             ...   
447                                52.0    2023.0 2022-12-26              PP   
448                                52.0    2023.0 2023-12-25              NP   
449                                52.0    2023.0 2023-12-25              PP   
450                                52.0    2024.0 2024-12-23              NP   
451                                52.0    2024.0 2024-12-23              PP   

business_program_type_cle

In [10]:
df_pivot['week_num'] = df_pivot['week_num'].astype(int)
df_pivot['year_num'] = df_pivot['year_num'].astype(int)
print(df_pivot)

business_program_type_cleaned  week_num  year_num       week activation_type  \
0                                     1      2022 2022-01-03              NP   
1                                     1      2022 2022-01-03              PP   
2                                     1      2023 2023-01-02              NP   
3                                     1      2023 2023-01-02              PP   
4                                     1      2024 2024-01-01              NP   
..                                  ...       ...        ...             ...   
447                                  52      2023 2022-12-26              PP   
448                                  52      2023 2023-12-25              NP   
449                                  52      2023 2023-12-25              PP   
450                                  52      2024 2024-12-23              NP   
451                                  52      2024 2024-12-23              PP   

business_program_type_cleaned  managed 

In [11]:
np_df = df_pivot[df_pivot.activation_type == "NP"]
pp_df = df_pivot[df_pivot.activation_type == "PP"]

In [12]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from joblib import load

launch_date = "2025-08-04"

np_df = df_pivot[df_pivot["activation_type"] == "NP"].copy()
pp_df = df_pivot[df_pivot["activation_type"] == "PP"].copy()

def add_lags(df):
    df = df.sort_values("week").reset_index(drop=True)
    df["organic_lag1"] = df["organic"].shift(1).fillna(method='bfill')
    df["organic_lag2"] = df["organic"].shift(2).fillna(method='bfill')
    return df

np_df = add_lags(np_df)
pp_df = add_lags(pp_df)

def predict_post_rewards(df, model_file, use_lag2=True):
    post_df = df[df["week"] >= launch_date].copy()
    if post_df.empty:
        print(f"No post-rewards data for {model_file}")
        return None
    
    #features = ["organic", "organic_lag1"]
    #if use_lag2:
        #features.append("organic_lag2")
    
    X_post = post_df["organic"]
    
    model = load(model_file)
    
    post_df["predicted_managed"] = model.predict(
        start=len(df[df["week"] < launch_date]),
        end=len(df)-1,
        exog=X_post
    )
    
    return post_df[["week", "managed", "predicted_managed"]]

np_post_preds = predict_post_rewards(np_df, "arimax_model_np.pkl", use_lag2=False)
pp_post_preds = predict_post_rewards(pp_df, "arimax_model_pp.pkl", use_lag2=True)

if np_post_preds is not None:
    np_post_preds.to_csv("np_post_rewards_predictions.csv", index=False)
    print("NP post-rewards predictions exported to np_post_rewards_predictions.csv")

if pp_post_preds is not None:
    pp_post_preds.to_csv("pp_post_rewards_predictions.csv", index=False)
    print("PP post-rewards predictions exported to pp_post_rewards_predictions.csv")

print("NP Post-Rewards Predictions:")
print(np_post_preds)

print("\nPP Post-Rewards Predictions:")
print(pp_post_preds)


ValueError: Provided exogenous values are not of the appropriate shape. Required (18, 1), got (15,).

In [ ]:
np_post_preds.head()

### Further edit

In [29]:
np_preds = np_post_preds.reset_index(drop= True)
pp_preds = pp_post_preds.reset_index(drop = True)

In [30]:
np_preds['addl_activations'] = ((np_preds['managed'] - np_preds['predicted_managed']).clip(lower=0)).astype(int)
pp_preds['addl_activations'] = ((pp_preds['managed'] - pp_preds['predicted_managed']).clip(lower=0)).astype(int)

In [32]:
np_preds

business_program_type_cleaned,week,managed,predicted_managed,addl_activations
0,2025-08-04,134,133.393125,0
1,2025-08-11,120,144.028087,0
2,2025-08-18,131,142.452876,0
3,2025-08-25,93,121.021250,0
4,2025-09-01,109,110.594195,0
5,2025-09-08,154,160.600157,0
6,2025-09-15,172,162.041984,9
7,2025-09-22,151,144.260809,6
8,2025-09-29,170,126.124932,43
9,2025-10-06,195,151.968766,43
